In [ ]:
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU langgraph==0.3.21 langchain-google-genai==2.1.2 langgraph-prebuilt==0.1.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 14.1 MB/s eta 0:00:00


In [ ]:
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableLambda

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

In [ ]:
class OrderState(dict):
    customer_name: str = ""
    coffee_type: str = ""
    size: str = ""
    confirmed: bool = False

In [ ]:
def greet(state):
    print("👋 Welcome to BaristaBot! What’s your name?")
    return state

def take_order(state):
    print(f"Hi {state.get('customer_name')}, what coffee would you like?")
    # Simulate input...
    state["coffee_type"] = "Espresso"
    state["size"] = "Medium"
    return state

def confirm_order(state):
    print(f"☕ Confirming: {state['size']} {state['coffee_type']} for {state['customer_name']}.")
    state["confirmed"] = True
    return state


In [ ]:
graph = StateGraph(OrderState)
graph.add_node("greet", greet)
graph.add_node("take_order", take_order)
graph.add_node("confirm_order", confirm_order)

graph.set_entry_point("greet")
graph.add_edge("greet", "take_order")
graph.add_edge("take_order", "confirm_order")
graph.add_edge("confirm_order", END)

app = graph.compile()

In [ ]:
order_state = OrderState(customer_name="Shakthi")
final_state = app.invoke(order_state)


👋 Welcome to BaristaBot! What’s your name?
Hi Shakthi, what coffee would you like?
☕ Confirming: Medium Espresso for Shakthi.


In [ ]:
from langgraph.graph import StateGraph, END

# Define the state
class OrderState(dict):
    customer_name: str = ""
    coffee_type: str = ""
    size: str = ""
    confirmed: bool = False

# Step 1: Greet the customer and ask for their name
def greet(state):
    state["customer_name"] = input("👋 Welcome to BaristaBot! What’s your name? ")
    return state

# Step 2: Ask for the coffee order
def take_order(state):
    state["coffee_type"] = input(f"Hi {state['customer_name']}, what coffee would you like? ")
    state["size"] = input("What size would you prefer (Small / Medium / Large)? ")
    return state

# Step 3: Confirm the order
def confirm_order(state):
    print(f"☕ Confirming: {state['size']} {state['coffee_type']} for {state['customer_name']}.")
    confirm = input("Is this correct? (yes/no): ").strip().lower()
    state["confirmed"] = confirm in ["yes", "y"]
    return state

# Step 4: Build the graph
graph = StateGraph(OrderState)
graph.add_node("greet", greet)
graph.add_node("take_order", take_order)
graph.add_node("confirm_order", confirm_order)

graph.set_entry_point("greet")
graph.add_edge("greet", "take_order")
graph.add_edge("take_order", "confirm_order")
graph.add_edge("confirm_order", END)

# Step 5: Run the bot
app = graph.compile()
final_state = app.invoke(OrderState())


👋 Welcome to BaristaBot! What’s your name? shakthi
Hi shakthi, what coffee would you like? expresso
What size would you prefer (Small / Medium / Large)? small
☕ Confirming: small expresso for shakthi.
Is this correct? (yes/no): yes


In [ ]:
from langgraph.graph import StateGraph, END

class OrderState(dict):
    customer_name: str = ""
    coffee_type: str = ""
    size: str = ""
    confirmed: bool = False
    exit: bool = False

# Menu to display
MENU = {
    "1": "Espresso",
    "2": "Cappuccino",
    "3": "Latte",
    "4": "Americano"
}

# Greet and get user name
def greet(state):
    print("👋 Welcome to BaristaBot!")
    state["customer_name"] = input("May I know your name? ")
    return state

# Show menu or quit
def main_menu(state):
    print(f"\nHi {state['customer_name']}! What would you like to do?")
    print("1. See Menu")
    print("2. Quit")
    choice = input("Enter your choice (1/2): ")

    if choice == "2":
        print("👋 Thank you! Have a great day!")
        state["exit"] = True
    return state

# Show coffee menu and take order
def take_order(state):
    print("\nHere's our coffee menu:")
    for key, item in MENU.items():
        print(f"{key}. {item}")
    coffee_choice = input("Please choose a coffee by number: ")

    if coffee_choice in MENU:
        state["coffee_type"] = MENU[coffee_choice]
    else:
        print("Invalid choice, selecting default: Espresso.")
        state["coffee_type"] = "Espresso"
    return state

# Ask for size
def choose_size(state):
    print(f"What size of {state['coffee_type']} would you like?")
    print("Options: Small / Medium / Large")
    state["size"] = input("Enter size: ").capitalize()
    return state

# Confirm the order
def confirm_order(state):
    print(f"\n☕ Order Summary for {state['customer_name']}:")
    print(f"{state['size']} {state['coffee_type']}")
    confirm = input("Confirm your order? (yes/no): ").lower()
    state["confirmed"] = confirm in ["yes", "y"]
    return state

# Build LangGraph
graph = StateGraph(OrderState)
graph.add_node("greet", greet)
graph.add_node("main_menu", main_menu)
graph.add_node("take_order", take_order)
graph.add_node("choose_size", choose_size)
graph.add_node("confirm_order", confirm_order)

# Entry and transitions
graph.set_entry_point("greet")
graph.add_edge("greet", "main_menu")
graph.add_conditional_edges(
    "main_menu",
    lambda s: "END" if s.get("exit") else "take_order"
)
graph.add_edge("take_order", "choose_size")
graph.add_edge("choose_size", "confirm_order")
graph.add_edge("confirm_order", END)

# Compile and run
app = graph.compile()
final_state = app.invoke(OrderState())


👋 Welcome to BaristaBot!
May I know your name? xyz

Hi xyz! What would you like to do?
1. See Menu
2. Quit
Enter your choice (1/2): 1

Here's our coffee menu:
1. Espresso
2. Cappuccino
3. Latte
4. Americano
Please choose a coffee by number: 2
What size of Cappuccino would you like?
Options: Small / Medium / Large
Enter size: small

☕ Order Summary for xyz:
Small Cappuccino
Confirm your order? (yes/no): yes


In [ ]:
import uuid  # For unique order ID
from langgraph.graph import StateGraph, END

class OrderState(dict):
    customer_name: str = ""
    coffee_type: str = ""
    size: str = ""
    confirmed: bool = False
    exit: bool = False
    order_id: str = ""

MENU = {
    "1": "Espresso",
    "2": "Cappuccino",
    "3": "Latte",
    "4": "Americano"
}

def greet(state):
    print("👋 Welcome to BaristaBot!")
    state["customer_name"] = input("May I know your name? ")
    return state

def main_menu(state):
    print(f"\nHi {state['customer_name']}! What would you like to do?")
    print("1. See Menu")
    print("2. Quit")
    choice = input("Enter your choice (1/2): ")

    if choice == "2":
        print("👋 Thank you! Have a great day!")
        state["exit"] = True
    return state

def take_order(state):
    print("\nHere's our coffee menu:")
    for key, item in MENU.items():
        print(f"{key}. {item}")
    coffee_choice = input("Please choose a coffee by number: ")

    if coffee_choice in MENU:
        state["coffee_type"] = MENU[coffee_choice]
    else:
        print("Invalid choice, selecting default: Espresso.")
        state["coffee_type"] = "Espresso"
    return state

def choose_size(state):
    print(f"What size of {state['coffee_type']} would you like?")
    print("Options: Small / Medium / Large")
    state["size"] = input("Enter size: ").capitalize()
    return state

def confirm_order(state):
    print(f"\n☕ Order Summary for {state['customer_name']}:")
    print(f"{state['size']} {state['coffee_type']}")
    confirm = input("Confirm your order? (yes/no): ").strip().lower()

    if confirm in ["yes", "y"]:
        state["confirmed"] = True
        state["order_id"] = str(uuid.uuid4())[:8]  # Short UUID
        print("\n🧾 ----- Your Coffee Bill -----")
        print(f"Order ID   : #{state['order_id']}")
        print(f"Name       : {state['customer_name']}")
        print(f"Order      : {state['size']} {state['coffee_type']}")
        print("------------------------------")
        print("🎉 Thank you for ordering with BaristaBot!\nEnjoy your drink ☕")
    else:
        state["confirmed"] = False
        print("\n❌ Order not confirmed.")
        print("Would you like to:")
        print("1. Reorder")
        print("2. Quit")
        retry = input("Enter your choice (1/2): ").strip()
        if retry == "2":
            state["exit"] = True
    return state


# LangGraph definition
graph = StateGraph(OrderState)
graph.add_node("greet", greet)
graph.add_node("main_menu", main_menu)
graph.add_node("take_order", take_order)
graph.add_node("choose_size", choose_size)
graph.add_node("confirm_order", confirm_order)

graph.set_entry_point("greet")
graph.add_edge("greet", "main_menu")
graph.add_conditional_edges("main_menu", lambda s: "END" if s.get("exit") else "take_order")
graph.add_edge("take_order", "choose_size")
graph.add_edge("choose_size", "confirm_order")
#graph.add_edge("confirm_order", END)
graph.add_conditional_edges(
    "confirm_order",
    lambda s: "END" if s.get("exit") else (
        "take_order" if not s.get("confirmed") else END
    )
)

app = graph.compile()
final_state = app.invoke(OrderState())


👋 Welcome to BaristaBot!

Hi abc! What would you like to do?
1. See Menu
2. Quit

Here's our coffee menu:
1. Espresso
2. Cappuccino
3. Latte
4. Americano
Invalid choice, selecting default: Espresso.
What size of Espresso would you like?
Options: Small / Medium / Large
